In [526]:
import pandas as pd
import math
from random import randint, seed
import json

import warnings
warnings.filterwarnings("ignore")

In [474]:
DATA = 'data'
seed(42)

# New sintetic data

In [475]:
desease = pd.read_csv(f'{DATA}/sintetic_deseases.csv', delimiter=';') #sasha directory
# desease = pd.read_csv('new_data.csv', delimiter=';')
desease = desease[desease['Column1']!='Болезнь']
desease = desease.rename(columns={
    'Column1': 'Диагноз',
    'Column2': 'Направление',
    'Column3': 'Обязательность',
    'Column4': 'Врач'
})
desease = desease[desease['Обязательность'].isin(('Обязательно', 'По показанию'))]

In [476]:
def format_doctor_name(doctor_name):
    if 'Врач-' in doctor_name:
        subsample = doctor_name.split('-')[1]
        subsample = subsample[0].upper() + subsample[1:]
        return subsample
    elif doctor_name == 'ЛОР-врач':
        return 'Отоларинголог'
    else: 
        return doctor_name

In [477]:
desease['Врач'] = desease['Врач'].map(format_doctor_name)

In [478]:
desease

,Диагноз,Направление,Обязательность,Врач
1,Астма,Спирометрия,Обязательно,Пульмонолог
2,Астма,Пикфлоуметрия,По показанию,Пульмонолог
3,Астма,Аллергические анализы,По показанию,Аллерголог
4,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог
5,Астма,Консультация пульмонолога,Обязательно,Пульмонолог
...,...,...,...,...
782,Сахарный диабет,Консультация эндокринолога,Обязательно,Эндокринолог
783,Артериальная гипертензия,Клинический осмотр,Обязательно,Врач общей практики
784,Артериальная гипертензия,Измерение артериального давления,Обязательно,Медсестра
785,Артериальная гипертензия,Общий анализ крови,Обязательно,Лаборант


In [479]:
desease['Врач'].unique()

array(['Пульмонолог', 'Аллерголог', 'Ортопед', 'Ревматолог',
       'Эндокринолог', 'Психиатр', 'Психолог', 'Терапевт',
       'УЗИ-специалист', 'Гематолог', 'Инфекционист', 'Отоларинголог',
       'Гастроэнтеролог', 'Невролог', 'Онколог', 'Иммунолог', 'Проктолог',
       'Дерматолог', 'Офтальмолог', 'Кардиолог', 'Педиатр', 'Гинеколог',
       'Генетик', 'Нефролог', 'Клинический лаборант',
       'Травматолог-ортопед', 'Логопед', 'Уролог', 'Врач общей практики',
       'Хирург', 'Нейрохирург', 'Миколог', 'Радиолог', 'Лаборант',
       'Инфекционист ', 'Специалист по электрофизиологии', 'Реабилитолог',
       'Врач реабилитолог', 'Рентгенолог', 'Ветеринар', 'Микробиолог',
       'Неврохирург', 'Патологоанатом', 'Оториноларинголог',
       'Физкультурник', 'Физиолог', 'Маммолог', 'Фтизиатр',
       'Колопроктолог', 'Психотерапевт', 'Вирусолог', 'Бактериолог',
       'Медсестра'], dtype=object)

In [480]:
doctors_freq = {}
for i in desease['Врач'].unique():
    doctors_freq[i] = {'n' : desease[desease['Врач']==i].shape[0],
                       'ids' : []
                      }
doctors_freq

{'Пульмонолог': {'n': 32, 'ids': []},
 'Аллерголог': {'n': 12, 'ids': []},
 'Ортопед': {'n': 18, 'ids': []},
 'Ревматолог': {'n': 18, 'ids': []},
 'Эндокринолог': {'n': 40, 'ids': []},
 'Психиатр': {'n': 30, 'ids': []},
 'Психолог': {'n': 12, 'ids': []},
 'Терапевт': {'n': 10, 'ids': []},
 'УЗИ-специалист': {'n': 10, 'ids': []},
 'Гематолог': {'n': 12, 'ids': []},
 'Инфекционист': {'n': 47, 'ids': []},
 'Отоларинголог': {'n': 9, 'ids': []},
 'Гастроэнтеролог': {'n': 80, 'ids': []},
 'Невролог': {'n': 99, 'ids': []},
 'Онколог': {'n': 31, 'ids': []},
 'Иммунолог': {'n': 5, 'ids': []},
 'Проктолог': {'n': 1, 'ids': []},
 'Дерматолог': {'n': 71, 'ids': []},
 'Офтальмолог': {'n': 9, 'ids': []},
 'Кардиолог': {'n': 29, 'ids': []},
 'Педиатр': {'n': 14, 'ids': []},
 'Гинеколог': {'n': 14, 'ids': []},
 'Генетик': {'n': 25, 'ids': []},
 'Нефролог': {'n': 2, 'ids': []},
 'Клинический лаборант': {'n': 1, 'ids': []},
 'Травматолог-ортопед': {'n': 4, 'ids': []},
 'Логопед': {'n': 1, 'ids': []},
 '

# Generate visits DataFrame

In [481]:
visits = pd.DataFrame(columns=desease.columns)

In [482]:
all_referrals = desease['Направление'].unique()
all_mandory = ['Обязательно', 'По показанию']

In [483]:
for doctor_i in list(desease['Врач'].unique()):
    doctor_info = desease[desease['Врач']==doctor_i]
    for desease_i in doctor_info['Диагноз'].unique():
        desease_info = doctor_info[doctor_info['Диагноз']==desease_i]
        true_referrals = desease_info.sample(desease_info.shape[0]-1)
        false_referrals = pd.DataFrame(columns=desease.columns)
        for referral_i in range(randint(0, 3)): # генерация неправильных направлеий
            new_referral = {
            'Диагноз': desease_i,
            'Направление': all_referrals[randint(1, len(all_referrals)-1)],
            'Обязательность': all_mandory[randint(0, 1)],
            'Врач': doctor_i,
            }
            false_referrals = pd.concat([false_referrals, pd.DataFrame(data=new_referral, index=[0], columns=desease.columns)])
        new_samples = pd.concat([true_referrals, false_referrals]).reset_index()
        new_samples = new_samples.drop(columns=['index'])
        visits = pd.concat([visits, new_samples])
visits = visits.reset_index().drop(columns=['index'])
print(visits.shape)
visits.head()

(971, 4)


,Диагноз,Направление,Обязательность,Врач
0,Астма,Пикфлоуметрия,По показанию,Пульмонолог
1,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог
2,Астма,Консультация пульмонолога,Обязательно,Пульмонолог
3,Аспергиллома,Биохимический анализ крови на уровень креатинина,Обязательно,Пульмонолог
4,Аспергиллома,Микроскопическое исследование пораженных волос,Обязательно,Пульмонолог


In [484]:
desease['Диагноз и Направление'] = desease[['Диагноз', 'Направление']].apply(tuple, axis=1)

## Some tests to check generation 

In [485]:
true_pairs = list(zip(list(desease['Диагноз']), list(desease['Направление'])))
true_and_false_pairs = list(zip(list(visits['Диагноз']), list(visits['Направление'])))
the_same = len(list(set(true_pairs) & set(true_and_false_pairs)))

print(f'всего правильных пар болезней и направлений {desease.shape[0]}')
print(f'в нашем датасете таких пар встречается {the_same}')

всего правильных пар болезней и направлений 786
в нашем датасете таких пар встречается 387


In [486]:
counter = 0
for pair in true_and_false_pairs:
    if pair in true_pairs:
        counter += 1
print(f'корректных назначений для болезней {counter} из {len(true_and_false_pairs)}')

корректных назначений для болезней 394 из 971


# Set ids for doctors

In [487]:
max_id = 1
for doctor in doctors_freq.keys():
    for i in range(math.ceil(doctors_freq[doctor]['n']/5)):
        doctors_freq[doctor]['ids'].append(max_id)
        max_id += 1

In [488]:
def set_id(doctor_name):
    possible_ids = doctors_freq[doctor_name]['ids']
    return randint(min(possible_ids), max(possible_ids))

In [489]:
visits['doctor_id'] = visits['Врач'].map(set_id)

In [490]:
visits['Диагноз и Направление'] = visits[['Диагноз', 'Направление']].apply(tuple, axis=1)

In [491]:
visits.head()

,Диагноз,Направление,Обязательность,Врач,doctor_id,Диагноз и Направление
0,Астма,Пикфлоуметрия,По показанию,Пульмонолог,1,"(Астма, Пикфлоуметрия)"
1,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог,3,"(Астма, Мониторинг показателей физической акти..."
2,Астма,Консультация пульмонолога,Обязательно,Пульмонолог,2,"(Астма, Консультация пульмонолога)"
3,Аспергиллома,Биохимический анализ крови на уровень креатинина,Обязательно,Пульмонолог,1,"(Аспергиллома, Биохимический анализ крови на у..."
4,Аспергиллома,Микроскопическое исследование пораженных волос,Обязательно,Пульмонолог,3,"(Аспергиллома, Микроскопическое исследование п..."


# Rate doctors

In [492]:
unique_ids = visits['doctor_id'].unique()
doctor_stats = {}
for doctor_id in unique_ids:
    test_doctor = visits[visits['doctor_id']==doctor_id]
    nesses = test_doctor[test_doctor['Обязательность']=='Обязательно']
    unnesses = test_doctor[test_doctor['Обязательность']=='По показанию']
    #print(type(doctor_id))
    print(test_doctor[test_doctor['doctor_id']==1])
    if doctor_id == 2:
        break
    true_nesses = 0
    all_nesses = 0
    for i in list(nesses['Диагноз и Направление']):
        nesses_right_healing = list(desease[desease['Обязательность']=='Обязательно'][desease['Диагноз']==i[0]]['Диагноз и Направление'])
        if i in nesses_right_healing:
            true_nesses += 1 
        all_nesses += len(nesses_right_healing)

    true_unnesses = 0
    all_unnesses = 0
    for i in list(unnesses['Диагноз и Направление']):
        unnesses_right_healing = list(desease[desease['Обязательность']=='По показанию'][desease['Диагноз']==i[0]]['Диагноз и Направление'])
        # print(unnesses_right_healing)
        if i in unnesses_right_healing:
            true_unnesses += 1 
        all_unnesses += len(unnesses_right_healing)

                                              Диагноз   
0                                               Астма  \
3                                        Аспергиллома   
5                                       Бронхоаденома   
13  Идиопатическая неспецифическая интерстициальна...   
25                  Острая интерстициальная пневмония   
26                  Острая интерстициальная пневмония   
27                  Острая интерстициальная пневмония   

                                          Направление Обязательность   
0                                       Пикфлоуметрия   По показанию  \
3    Биохимический анализ крови на уровень креатинина    Обязательно   
5                       Изучение симптомов и анамнеза    Обязательно   
13               Бронхоальвеолярная лаважная жидкость   По показанию   
25  Рентгенография или компьютерная томография гру...    Обязательно   
26                              Электронейромиография   По показанию   
27            Бактериологическое исслед

In [493]:
all_unnesses

5

### Подсчет для одного доктора

In [494]:
unique_ids = visits['doctor_id'].unique()
doctor_stats = {}
for doctor_id in unique_ids:
    test_doctor = visits[visits['doctor_id']==doctor_id]
    nesses = test_doctor[test_doctor['Обязательность']=='Обязательно']
    unnesses = test_doctor[test_doctor['Обязательность']=='По показанию']

In [495]:
doctor_1 = visits[visits['doctor_id']==2]

In [496]:
doctor_1.head()

,Диагноз,Направление,Обязательность,Врач,doctor_id,Диагноз и Направление
2,Астма,Консультация пульмонолога,Обязательно,Пульмонолог,2,"(Астма, Консультация пульмонолога)"
11,Идиопатическая неспецифическая интерстициальна...,Клиническое обследование,Обязательно,Пульмонолог,2,(Идиопатическая неспецифическая интерстициальн...
12,Идиопатическая неспецифическая интерстициальна...,Рентгенография грудной клетки,Обязательно,Пульмонолог,2,(Идиопатическая неспецифическая интерстициальн...
19,Кавернозный туберкулёз,Компьютерная томография грудной клетки,Обязательно,Пульмонолог,2,"(Кавернозный туберкулёз, Компьютерная томограф..."
29,Рак легкого,Анализ крови на наличие аутоантител,По показанию,Пульмонолог,2,"(Рак легкого, Анализ крови на наличие аутоанти..."


In [497]:
y_true_nesses = [0]*len(desease)
y_pred_nesses = [0]*len(desease)
y_true_unnesses = [0]*len(desease)
y_pred_unnesses = [0]*len(desease)

#заполнение y_true_nesses
for i in range(len(desease)):
    for k in range(len(nesses)):
        if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
            y_true_nesses[i] = 1

#заполнение y_pred_nesses
for i in range(len(desease)):
    for k in range(len(nesses)):
        if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and nesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
            #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
            y_pred_nesses[i] = 1

#заполнение y_true_unnesses
for i in range(len(desease)):
    for k in range(len(unnesses)):
        if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
            y_true_unnesses[i] = 1

#заполнение y_pred_unnesses
for i in range(len(desease)):
    for k in range(len(unnesses)):
        if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and unnesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
            #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
            y_pred_unnesses[i] = 1

In [498]:
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score

In [499]:
nesses_score = fbeta_score(y_true_nesses, y_pred_nesses, average='macro', beta=2)
unnesses_score = fbeta_score(y_true_unnesses, y_pred_unnesses, average='macro', beta=0.5)

In [500]:
unnesses_score

0.4979623025980641

In [501]:
SCORE = (5*nesses_score + unnesses_score)/6

In [502]:
SCORE

0.9163270504330107

In [503]:
def count_score_doctor(visits: pd.DataFrame) -> dict:
    unique_ids = visits['doctor_id'].unique()
    doctor_stats = {}
    for doctor_id in unique_ids:
        test_doctor = visits[visits['doctor_id']==doctor_id]
        nesses = test_doctor[test_doctor['Обязательность']=='Обязательно']
        unnesses = test_doctor[test_doctor['Обязательность']=='По показанию']
        #doctor = visits[visits['doctor_id']==doctor_id]

        y_true_nesses = [0]*len(desease)
        y_pred_nesses = [0]*len(desease)
        y_true_unnesses = [0]*len(desease)
        y_pred_unnesses = [0]*len(desease)
        
        #заполнение y_true_nesses
        for i in range(len(desease)):
            for k in range(len(nesses)):
                if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
                    y_true_nesses[i] = 1
        
        #заполнение y_pred_nesses
        for i in range(len(desease)):
            for k in range(len(nesses)):
                if nesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and nesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
                    #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
                    y_pred_nesses[i] = 1
        
        #заполнение y_true_unnesses
        for i in range(len(desease)):
            for k in range(len(unnesses)):
                if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i]:
                    y_true_unnesses[i] = 1
        
        #заполнение y_pred_unnesses
        for i in range(len(desease)):
            for k in range(len(unnesses)):
                if unnesses['Диагноз'].iloc[k] == desease['Диагноз'].iloc[i] and unnesses['Направление'].iloc[k] == desease['Направление'].iloc[i]:
                    #print(desease['Диагноз'].iloc[i], desease['Направление'].iloc[i])
                    y_pred_unnesses[i] = 1
                    
        nesses_score = fbeta_score(y_true_nesses, y_pred_nesses, average='macro', beta=2)
        unnesses_score = fbeta_score(y_true_unnesses, y_pred_unnesses, average='micro', beta=0.5)
        SCORE = (5*nesses_score + unnesses_score)/6*100
        doctor_stats[doctor_id] = SCORE
    return doctor_stats

In [504]:
count_score_doctor(visits)

{1: 66.38006774551232,
 3: 58.08921145623865,
 2: 69.06207007540785,
 7: 67.9561902667642,
 6: 68.14702996142069,
 4: 70.43924927422468,
 5: 99.93638676844783,
 10: 99.8303647158609,
 9: 70.43924927422468,
 8: 58.16352468661462,
 14: 74.08303458645248,
 11: 65.5733056037624,
 12: 72.64474973436793,
 13: 68.04100790883373,
 18: 68.12651396143396,
 17: 67.91262495044941,
 15: 68.10462114038592,
 16: 63.11370564884808,
 20: 70.3527466439129,
 26: 70.79605675537067,
 19: 58.17402909830821,
 23: 64.9195568111238,
 22: 99.93638676844783,
 24: 64.7228858282444,
 25: 63.89054867157743,
 21: 62.74594907407408,
 29: 67.91262495044941,
 30: 67.8932654457427,
 27: 61.77471026947554,
 32: 68.55369895501154,
 31: 64.12379718726869,
 28: 69.11728351348651,
 34: 62.15617586771452,
 35: 60.14373508794978,
 33: 58.08763447139449,
 36: 58.17402909830821,
 37: 61.59786279336805,
 39: 65.32592878444493,
 38: 57.856223294649055,
 42: 77.15592396023806,
 41: 70.38665521059188,
 40: 69.22330556607346,
 51: 67

In [505]:
desease.to_csv()

',Диагноз,Направление,Обязательность,Врач,Диагноз и Направление\n1,Астма,Спирометрия,Обязательно,Пульмонолог,"(\'Астма\', \'Спирометрия\')"\n2,Астма,Пикфлоуметрия,По показанию,Пульмонолог,"(\'Астма\', \'Пикфлоуметрия\')"\n3,Астма,Аллергические анализы,По показанию,Аллерголог,"(\'Астма\', \'Аллергические анализы\')"\n4,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог,"(\'Астма\', \'Мониторинг показателей физической активности\')"\n5,Астма,Консультация пульмонолога,Обязательно,Пульмонолог,"(\'Астма\', \'Консультация пульмонолога\')"\n6,Остеоартрит,Клиническое обследование суставов,Обязательно,Ортопед,"(\'Остеоартрит\', \'Клиническое обследование суставов\')"\n7,Остеоартрит,Рентгеновское исследование суставов,Обязательно,Ортопед,"(\'Остеоартрит\', \'Рентгеновское исследование суставов\')"\n8,Остеоартрит,МРТ суставов,По показанию,Ревматолог,"(\'Остеоартрит\', \'МРТ суставов\')"\n9,Остеоартрит,Анализ крови на наличие воспалительных маркеров,По показанию,Ревматолог

# Set clients id

In [506]:
from sklearn.preprocessing import LabelEncoder

In [507]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [508]:
visits['Диагноз_copy'] = visits['Диагноз']
visits = MultiColumnLabelEncoder(columns = ['Диагноз_copy']).fit_transform(visits)
visits = visits.rename(columns={'Диагноз_copy':'client_id'})

In [533]:
visits

,Диагноз,Направление,Обязательность,Врач,doctor_id,Диагноз и Направление,client_id,doctors_and_clients
0,Астма,Пикфлоуметрия,По показанию,Пульмонолог,1,"(Астма, Пикфлоуметрия)",20,"(1, 20)"
1,Астма,Мониторинг показателей физической активности,По показанию,Пульмонолог,3,"(Астма, Мониторинг показателей физической акти...",20,"(3, 20)"
2,Астма,Консультация пульмонолога,Обязательно,Пульмонолог,2,"(Астма, Консультация пульмонолога)",20,"(2, 20)"
3,Аспергиллома,Биохимический анализ крови на уровень креатинина,Обязательно,Пульмонолог,1,"(Аспергиллома, Биохимический анализ крови на у...",19,"(1, 19)"
4,Аспергиллома,Микроскопическое исследование пораженных волос,Обязательно,Пульмонолог,3,"(Аспергиллома, Микроскопическое исследование п...",19,"(3, 19)"
...,...,...,...,...,...,...,...,...
966,Грипп,Консультация невролога или генетика,Обязательно,Вирусолог,181,"(Грипп, Консультация невролога или генетика)",110,"(181, 110)"
967,ОРВИ,Антигельминтная терапия,По показанию,Вирусолог,181,"(ОРВИ, Антигельминтная терапия)",186,"(181, 186)"
968,Артериальная гипертензия,Аллерготесты,По показанию,Медсестра,183,"(Артериальная гипертензия, Аллерготесты)",17,"(183, 17)"
969,Артериальная гипертензия,УЗИ слюнных желез,По показанию,Медсестра,183,"(Артериальная гипертензия, УЗИ слюнных желез)",17,"(183, 17)"


In [538]:
visits['doctor_id'] = visits['doctor_id'].map(str)
visits['client_id'] = visits['client_id'].map(str)

In [512]:
visits.to_csv('visits_with_client_ids.csv')

# Make function for predict metric from json

Передаю:
массив докторов
У каждого доктора, есть массив пациентов
У каждого пациента, его диагноз и массив назначений
У назначения наименование назначения

Хочу: массив id докторов
У каждого доктора его Rate от 0 - 10
И массив id пациентов
У каждого пациента массив направлений
У направления есть id
И state где 
1 что все правильно
2 не сделал
3 лишнее

In [ ]:
input_example = 
{
    'doctors_ids': [1, 2],
    'doctors_to_client': {1: 20, 2: 17},
    'clients':  {20: {'desease': 'Астма',
                      'referrals': # которые выписал ему доктор
                        { 'По показанию': ['Пикфлоуметрия'],
                          'Обязательно': ['Мониторинг показателей физической активности']}
                     },
                 17: {'desease': 'Артериальная гипертензия',
                      'referrals':
                       { 'По показанию': ['Аллерготесты'], 
                         'Обязатально': ['УЗИ слюнных желез']}
                     }
                }
}

output_example = 
{
    'doctor_rates':
    {
        157: 5.8,  #doctor_id - rate
        161: 5.7,
        162: 5.9,
    }
    'referrals_info':
    {
        20: 
        {
            'Пикфлоуметрия': 2,
            'Мониторинг показателей физической активности': 1,
            'Микроскопическое исследование пораженных волос': 3
        },
        17:
        {
            'Консультация невролога или генетика': 3, 
            'УЗИ слюнных желез': 1,
            'Пульсоксиметрия': 1
        }
    }
}

In [539]:
def transform_pandas_to_json(visits):
    input_json = {}
    doctor_ids = list(visits['doctor_id'].unique())
    clients_ids = list(visits['client_id'].unique())
    input_json['doctors_ids'] = doctor_ids
    visits['doctors_and_clients'] = visits[['doctor_id', 'client_id']].apply(tuple, axis=1)
    doctors_and_clients = list(visits['doctors_and_clients'])
    input_json['doctors_to_client'] = {}
    for i in doctors_and_clients:
        input_json['doctors_to_client'][i[0]] = i[1]
    input_json['clients'] = {}
    for i in clients_ids:
        input_json['clients'][i] = {}
        desease_name = list(visits[visits['client_id']==i]['Диагноз'])[0]
        input_json['clients'][i]['desease'] = desease_name
        mandatory_yes = list(visits[visits['client_id']==i][visits['Обязательность']=='Обязательно']['Направление'])
        mandatory_no = list(visits[visits['client_id']==i][visits['Обязательность']=='По показанию']['Направление'])
        input_json['clients'][i]['referrals'] = {}
        input_json['clients'][i]['referrals']['По показанию'] = mandatory_no
        input_json['clients'][i]['referrals']['Обязательно'] = mandatory_yes
    return input_json

In [542]:
input_json

{'doctors_ids': ['1',
  '3',
  '2',
  '7',
  '6',
  '4',
  '5',
  '10',
  '9',
  '8',
  '14',
  '11',
  '12',
  '13',
  '18',
  '17',
  '15',
  '16',
  '20',
  '26',
  '19',
  '23',
  '22',
  '24',
  '25',
  '21',
  '29',
  '30',
  '27',
  '32',
  '31',
  '28',
  '34',
  '35',
  '33',
  '36',
  '37',
  '39',
  '38',
  '42',
  '41',
  '40',
  '51',
  '46',
  '44',
  '45',
  '50',
  '43',
  '48',
  '52',
  '47',
  '49',
  '53',
  '54',
  '64',
  '70',
  '62',
  '67',
  '69',
  '57',
  '56',
  '60',
  '68',
  '61',
  '65',
  '59',
  '66',
  '58',
  '63',
  '55',
  '90',
  '81',
  '76',
  '83',
  '80',
  '89',
  '87',
  '88',
  '86',
  '71',
  '82',
  '74',
  '84',
  '79',
  '78',
  '72',
  '75',
  '77',
  '85',
  '73',
  '96',
  '97',
  '92',
  '91',
  '95',
  '94',
  '93',
  '98',
  '104',
  '103',
  '112',
  '101',
  '105',
  '102',
  '111',
  '107',
  '110',
  '100',
  '109',
  '113',
  '114',
  '106',
  '108',
  '116',
  '115',
  '121',
  '117',
  '118',
  '119',
  '120',
  '122',
  '

In [544]:
input_json = transform_pandas_to_json(visits)
with open("input_visits.json", "w", encoding='utf8') as outfile:
    json.dump(input_json, outfile, ensure_ascii=False)